# Derenning

These exercises make use of the PyNeb software (Luridiana et al. 2015)

All observations are affected by extinction due to interstellar dust. The first thing an observer must do
once the spectra are calibrated and the line intensities measured is to correct the line intensities for extinction. Published data nowadays often give only reddening-corrected line fluxes and the value of the extinction in the
V band, A$_V$. 
It may however be of interest to recover the observed intensities and apply a different extinction law.

## The effect of different reddening laws

### Take the reddening-corrected line intensities of N44C, a compact H II region in the Large Magellanic Cloud, from Table 3 in Garnett et al. (2000). For this object, assuming an intrinsic H$\alpha$/H$\beta$ ratio equal to 2.86 the authors find A$_V$ = 0.25 using the classical Cardelli et al. (1989) extinction law with R$_V$ = 3.1.

##### We first tell the notebook to send figures inside the document, not in separate windows. We then import some needed libraries, with aliases.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pyneb as pn

In [ ]:
pn.__version__

##### Consider the deep spectrum of the Galactic \hii\ region NGC 3603 obtained by Garc\'ia Rojas et al. (2006) (their Table 2).
##### See http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J/MNRAS/368/253&-to=3
##### The table can be downloaded in ; separated values.

##### We read the file using the numpy genfromtxt utility:

In [ ]:
data = np.genfromtxt('../Data/Garcia-Rojas.tsv', 
                     delimiter=';', 
                     dtype=('U8', 'float', 'U9', 'U15', 'float', 'float', 'float', 'int', 'a1','a1'), 
                     skip_header=47, 
                     names=('Cluster','Lambda0','Ion','Mult','LambdaObs','F(Lambda)','I(Lambda)','e_F(Lambda)','u_F(Lambda)','Notes'))

##### Let's have a look at the __data__ variable.

In [ ]:
print(data[0])
print(len(data))

In [ ]:
print(data['Cluster'][0:10])

In [ ]:
print(np.unique(data['Cluster']))

##### We will only use the M 16 data

In [ ]:
data = data[data['Cluster'] == 'NGC 3603']
print(len(data))

### Determine the redenning correction from H$\beta$/H$\alpha$.

##### We instantiate a RedCorr object from the PyNeb library. We then print the list of available extinction laws, using a method (function) of the object.

In [ ]:
rc = pn.RedCorr()
rc.getLaws()

In [ ]:
rc.printLaws()

##### We select the Cardelli et al law.

##### We define the value of visual attenuation.

##### Using the Fitzpatrick (1999) extinction law with  $R_V=3.1$, compute the reddening-corrected intensities for the hydrogen lines and for all the forbidden lines assuming an intrinsic value for H$\alpha$/H$\beta$ of 2.83.

In [ ]:
rc.law = 'F99'
rc.R_V = 3.1

In [ ]:
data.dtype.names

In [ ]:
print(data[data['Lambda0'] == 6562.82]['FLambda'][0] / 100)

In [ ]:
obs_ratio = data[data['Lambda0'] == 6562.82]['FLambda'][0] / 100
rc.setCorr(obs_ratio/2.83, 6563., 4861.)

##### We can print out the values of E$_{B-V}$ and cH$\beta$.

In [ ]:
print(rc.E_BV)
print(rc.cHbeta)
print(rc.AV)

##### We get the extinction relative to H$\beta$ for the wavelength vector.

In [ ]:
print(rc.getCorrHb(6563))
print(rc.getCorrHb(6563) * 6.1488)

In [ ]:
correction = rc.getCorrHb(data['Lambda0'])
print(correction)

##### We make a copy of the original data not to loose them. We apply the correction to the new data set. As we want to correct for extinction, we divide by the values obtained from PyNeb.

In [ ]:
data2 = data.copy()
# data2['intens'] = data2['intens'] / extinction
data2['FLambda'] *= correction

##### We can print all the corrected intensities and compare with what Garci-Rojas obtained.

In [ ]:
print('Line      wl  uncor. intensity')
for d2 in data2:
    print('{:8s} {:4.0f} {:.3f} {:.3f}'.format(d2['Ion'], 
                                               d2['Lambda0'], 
                                               d2['FLambda'],
                                               d2['ILambda']))

### Do the same, but for $R_V=5$.

##### We change the extinction law and the value of R$_V$. We store the observed value of H$\alpha$/H$\beta$.

In [ ]:
rc.R_V = 5.0

##### We define a new correction using the observed and theoritical values of H$\alpha$/H$\beta$ and the vector of wavelengths.

In [ ]:
rc.setCorr( obs_ratio/ 2.83, 6563., 4861)
print(rc.AV)

In [ ]:
correction2 = rc.getCorrHb(data2['Lambda0'])
print(correction2)

##### We now correct the data sample from the extinction.

In [ ]:
data3 = data.copy()
data3['FLambda'] *= correction2

##### We print the comparison between results obtained applying the two corrections.

In [ ]:
print('Line      wl    RV=3.1  5.0 GS')
for d2, d3 in zip(data2, data3):
    print('{:8s} {:4.0f} {:.3f} {:.3f} {:.3f}'.format(d2['Ion'], 
                                               d2['Lambda0'], 
                                               d2['FLambda'], 
                                               d3['FLambda'],
                                              d3['ILambda']))

### Compute the reddening-corrected values of  H$\gamma$/H$\beta$, H$\delta$/H$\beta$, [OIII]$\lambda$4363/[OIII]$\lambda$5007, and [OII]$\lambda$3727/[OII]$\lambda$7325 for both cases of $R_V$.

### ??? Comment on the results.

### Plot the different extinction laws using the PyNeb RedCorr plot facility.

In [ ]:
f, axes =plt.subplots(2, 1, figsize=(10,16))
rc.R_V = 3.1
rc.plot(laws=['F99', 'CCM89'], ax=axes[0])
axes[0].set_title('R_V = 3.1')
rc.R_V = 5
rc.plot(laws=['F99', 'CCM89'], ax=axes[1])
axes[1].set_title('R_V = 5.0')

In [1]:
# the following is to have the nice style in the Notebook.
# Don't remove this.
# You will need a directory named style containing the custom.css file, 
# downlodable from the same place than this notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()